**Question** Build, compile and train flowers dataset using transfer learning (Resnet50) and data augmentation



**Description**:

Load the flowers training and testing datasets in respective paths 

The data set already splitted into 80:20 ratio

Get the pre-trained resnet50 model and freeze initial layers.

For fully connected layers use 512 neurons and 5 neurons in the last layer having relu and softmax activation functions respectively

Compile the model with loss as categorical cross-entropy and optimizer as stochastic gradient descent

Using Image data generators preprocess train and testing sets by including augmentation in training data generator. Make sure to use (224,224) as image size or shape

Train the model with 5 epochs, 64 as a batch size


**Level**: Hard


**Input format**: Dataset


**Output format**: Accuracy


**Sample Input**: Flowers dataset


**Sample Output**: Accuracy: 0.8625298142433167


**SOLUTION**:


In [ ]:
import os
import shutil
import numpy as np
import glob   
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Input, Add, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D

# set to where the 'flowers' directory is located
data_dir = '/home/metagogy/Downloads/flowers'

# Training data dir
training_dir = '/home/metagogy/Downloads/flowers/Train'

# Test data dir
testing_dir = '/home/metagogy/Downloads/flowers/Test'

# number of classes 
num_classes = 5

def get_model():
    base_model = ResNet50(weights='imagenet', include_top=False)
    for layer in base_model.layers:
        layer.trainable = False 
    base_model_ouput = base_model.output
    x = GlobalAveragePooling2D()(base_model_ouput)
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

# Get the model
model = get_model()

# compile 
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Using ImageDataGenerator for pre-processing

image_size = 224
batch_size = 64

# training data
train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input, 
                                    shear_range=0.2, zoom_range=0.2, 
                                    horizontal_flip=True)

# validation data 
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)

# create data generator objects
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

epochs = 5

# training
model.fit_generator( 
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)
